In [ ]:
import numpy as np
import pandas as pd
import pickle
import json
import plotly.graph_objects as go
import plotly
from plotly.subplots import make_subplots
import plotly.express as px
from scipy.stats import *


NEMATODES = ['nematode_0','nematode_1','nematode_2','nematode_3','nematode_4','nematode_5']

COLOR_PALETTE = ["#cd34b5","#ab7ca3","#0064ab"]

COLOR_PALETTE_COMMS = ['#d0e3f5',
                       '#d60000',
                       '#8c3bff',
                       '#018700',
                       '#00acc6',
                       '#e6a500',
                       '#ff7ed1',
                       '#6b004f',
                       '#573b00',
                       '#005659',
                       '#15e18c',
                       "#ffd700",
                       "#ffb14e",
                       "#fa8775",
                       "#ea5f94",
                       "#cd34b5",
                       "#9d02d7",
                       "#ab7ca3",
                       "#0064ab",
                       "#0000ff"]

In [32]:
import scipy.stats as st

In [ ]:
def load_curve(filename,dataset):
    
    return(np.load(dataset + '/pointClouds/' + filename + '.npy'))

In [ ]:
with open('communities_nematodes_matroidgenerators.json', 'r') as f:
    COMMUNITIES_nematodes =json.load(f)

with open('communities_zebrafish_matroidgeneratos.json', 'r') as f:
    COMMUNITIES_zebrafish =json.load(f)

In [ ]:
def load_centrality(filename,dataset,generators):
    cc = pd.read_csv(dataset + '/' + generators+ '_generators' + '/nodeCents/' + filename + '.tsv', sep="\t", header=None).values
    for i in range(len(cc),len(load_curve(filename,dataset))):
        cc = np.append(cc,0)
    return cc



In [ ]:
def plot_chain_with_centrality(filename, dataset, generators = 'matroid'):
    
    curve = load_curve(filename,dataset)
    nodes = load_centrality(filename,dataset, 'matroid')

    if dataset == 'nematodes':
        df = pd.DataFrame.from_records(curve[:,:2], columns=['X', 'Y'])
        df["node centrality"] = nodes    
        fig = px.scatter(
            df, x="X", y="Y", 
            color="node centrality",
        )
        fig = go.Figure(fig)
        fig.update_traces(
            mode='markers + lines',
            marker=dict(size=10,
                        line=dict(width=0.01,
                                color='DarkSlateGrey')
                       ),
            line_width = 2
        )

        fig.update_traces(hoverinfo="text",hovertemplate=[i for i in range(len(curve))],
                     )
        fig.update_layout(showlegend=False)
        fig.update_layout(coloraxis_showscale=False)
        fig.update_layout(paper_bgcolor="white",
                 plot_bgcolor="white",
                     width=500, 
                  height=500)
        fig.update_xaxes(showticklabels=False,
                    title = ' ')
        fig.update_yaxes(showticklabels=False,
                    title = ' ')
        
    else:
        fig = go.Figure()
        df = pd.DataFrame.from_records(curve, columns=['X', 'Y','Z'])
        df["node centrality"] = nodes    
        fig.add_trace(go.Scatter3d(
        x=df['X'], y=df['Y'], z=df['Z'], 
        name = filename,    
        marker=dict(
            size=3,
            color = df["node centrality"],
            line=dict(width=0.01, 
                      color='DarkSlateGrey')

        ),
        line=dict(
            width=8,
            color = df["node centrality"],
        )),
        )
        fig.update_layout(scene=dict(xaxis = dict(
                             backgroundcolor="rgb(200, 200, 230)",
                             gridcolor='rgba(0,0,0,0)',
                             showbackground=False,
                             zerolinecolor='rgba(0,0,0,0)',showticklabels=False,),
        yaxis = dict(
                             backgroundcolor="rgb(200, 200, 230)",
                             gridcolor='rgba(0,0,0,0)',
                             showbackground=False,
                             zerolinecolor='rgba(0,0,0,0)',showticklabels=False),
                   zaxis = dict(
                             backgroundcolor="rgb(200, 200, 230)",
                             gridcolor='rgba(0,0,0,0)',
                             showbackground=False,
                             zerolinecolor='rgba(0,0,0,0)',showticklabels=False),
        ))

        fig.update_layout(scene = dict(
                        xaxis_title=' ',
                        yaxis_title=' ',
                        zaxis_title=' '))
    return(fig)






In [ ]:
def communities_color_code(comms):
    n = np.sum([len(comms[k]) for k in comms.keys()])
    nodes = n*[0]
    color = 1
    for k in comms.keys():
        if len(comms[k])>1:
            for j in comms[k]:
                nodes[j] = color
            color = color+1
    return nodes


def plot_chain_with_communities(filename, dataset):
    
    curve = load_curve(filename,dataset)
    if dataset == 'nematodes':
        community = COMMUNITIES_nematodes[filename]
        df = pd.DataFrame.from_records(curve[:,:2], columns=['X', 'Y'])
        nodes = communities_color_code(community)
        df["node centrality"] = nodes    
        fig = px.scatter(
            df, x="X", y="Y", 
            color="node centrality",
            color_continuous_scale= COLOR_PALETTE_COMMS
        )
        fig = go.Figure(fig)
        fig.update_traces(
            mode='markers + lines',
            marker=dict(size=10,
                        line=dict(width=0.01,
                                color='DarkSlateGrey')
                       ),
            line_width = 2
        )
        fig.update_traces(hoverinfo="text",hovertemplate=nodes)  
        fig.update_layout(showlegend=False)
        fig.update_layout(coloraxis_showscale=False)
        fig.update_layout(paper_bgcolor="white",
                     plot_bgcolor="white",
                        width=500, 
                      height=500)
        fig.update_xaxes(showticklabels=False,
                        title = ' ')
        fig.update_yaxes(showticklabels=False,
                        title = ' ')
        
    else:
        community = COMMUNITIES_zebrafish[filename]
        df = pd.DataFrame.from_records(curve, columns=['X', 'Y','Z'])
        nodes = communities_color_code(community)
        df["node centrality"] = nodes    
        fig = go.Figure()
        fig.add_trace(go.Scatter3d(
        x=df['X'], y=df['Y'], z=df['Z'], 
        name = 'Curve',    
        marker=dict(
            size=8,
            color = df["node centrality"],
            colorscale=COLOR_PALETTE,
            line=dict(width=4, 
                      color='DarkSlateGrey')
        ),
        line=dict(
            width=8,
            color = df["node centrality"],
            colorscale=COLOR_PALETTE
        )),
        )
        fig.update_layout(scene=dict(xaxis = dict(
                             backgroundcolor="rgb(200, 200, 230)",
                             gridcolor='rgba(0,0,0,0)',
                             showbackground=False,
                             zerolinecolor='rgba(0,0,0,0)',showticklabels=False,),
        yaxis = dict(
                             backgroundcolor="rgb(200, 200, 230)",
                             gridcolor='rgba(0,0,0,0)',
                             showbackground=False,
                             zerolinecolor='rgba(0,0,0,0)',showticklabels=False),
                   zaxis = dict(
                             backgroundcolor="rgb(200, 200, 230)",
                             gridcolor='rgba(0,0,0,0)',
                             showbackground=False,
                             zerolinecolor='rgba(0,0,0,0)',showticklabels=False),
        ))

        fig.update_layout(scene = dict(
                        xaxis_title=' ',
                        yaxis_title=' ',
                        zaxis_title=' '))
        fig.update_traces(hoverinfo="text",hovertemplate=nodes)
        
    return(fig)

# HyperTDA detects and quantifies bias in zebrafish in vivo tracks

## Matroid generators

In [ ]:
DDf_b_o = pd.read_csv('communities_overlap_zebrafish_matroidgeneratos.csv')
Df = pd.read_csv('communities_stat_zebrafish_matroidgeneratos.csv')

In [ ]:
DDf_b_o

In [ ]:
def number_of_comms(group):
    """
    Computes number of total and 'True' communities in a partition. 
    """    
    
    
    count = 0
    count_all = 0
    which = []
    for el in group.keys():
        count_all = count_all+1
        if len(group[el])>1:
            count = count+1
            which.append(group[el])
    return [count, which, count_all]    

def analyse_overlap(group):
    """
    Quantifies the pairwise overlap between communities in a partition,
    as the p-value of the Mann-Whitney U-test
    """
    
    l,cs,l2 = number_of_comms(group)
    M = []
    for i in range(len(cs)):
        for j in range(i+1,len(cs)):
            U1, p = mannwhitneyu(cs[i],cs[j])
            M.append(p)
    M = np.array(M)
    return M    


In [ ]:
fig = px.box(DDf_b_o, x ='Overlap', color = 'Type',
             log_x=True,
               title= 'Overlap of communities',
             notched = True,
            hover_name='Name',
            color_discrete_sequence=COLOR_PALETTE)

fig.show()

In [ ]:

Df['Partial ambient ass'] = Df['Size']/Df['ROG']
Df['Partial intrinsic ass'] = Df['Size']/Df['Life Span']

In [ ]:
fig = px.box(Df, x ='Life Span', color = 'Type',
               title= 'Geodesic distance',
             notched = True,
            hover_name='Name',
            color_discrete_sequence=COLOR_PALETTE)

fig.show()

In [ ]:
fig = px.box(Df, x ='ROG', color = 'Type',
               title= 'Volume',
             notched = True,
            hover_name='Name',
            color_discrete_sequence=COLOR_PALETTE)

fig.show()

In [ ]:
fig = px.box(Df, x ='Size', color = 'Type',
               title= 'Size of communities',
             notched = True,
            hover_name='Name',
            color_discrete_sequence=COLOR_PALETTE)

fig.show()

In [ ]:
vectors = []
s = Df.loc[Df['Type'] == 'unbiased']
ss = DDf_b_o.loc[DDf_b_o['Type'] == 'unbiased']
o = np.array(ss['Overlap'])
r = np.array(s['ROG'])
w = np.array(s['Size'])
l = np.array(s['Life Span'])  
w = np.log(w)


    
vectors.append([np.percentile(r, 50),np.percentile(w, 50),np.percentile(l, 50),10*np.percentile(o, 50)])



s = Df.loc[Df['Type'] == 'zebrafish']
ss = DDf_b_o.loc[DDf_b_o['Type'] == 'zebrafish']
o = np.array(ss['Overlap'])
r = np.array(s['ROG'])
w = np.array(s['Size'])
l = np.array(s['Life Span'])  
w = np.log(w)

vectors.append([np.percentile(r, 50),np.percentile(w, 50),np.percentile(l, 50),10*np.percentile(o, 50)])

s = Df.loc[Df['Type'] == 'biased']
ss = DDf_b_o.loc[DDf_b_o['Type'] == 'biased']
o = np.array(ss['Overlap'])
r = np.array(s['ROG'])
w = np.array(s['Size'])
l = np.array(s['Life Span'])  
w = np.log(w)

vectors.append([np.percentile(r, 50),np.percentile(w, 50),np.percentile(l, 50),10*np.percentile(o, 50)])



vectors

In [ ]:
fig = go.Figure()

for i in [0,2,1]:
    fig.add_trace(go.Scatterpolar(
        r= vectors[i],
        theta = ['ROG', 'Weight', 'Life span', 'Separation'],
        name = ['Unbiased tracks' ,'Biased tracks','Zebrafish tracks'][i],
        fill='toself',
        marker=dict(color = COLOR_PALETTE[i])
    ))


    
fig.update_layout(coloraxis_showscale=False)
fig.update_layout(paper_bgcolor="white",
                 plot_bgcolor="white",
                    width=500, 
                  height=500)
fig.update_xaxes(showticklabels=False,
                    title = ' ')
fig.update_yaxes(showticklabels=False,
                    title = ' ')

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=False
    ),
  ),
)

#fig.write_image('radar_matroid.svg')


fig.show()    

In [ ]:
# bar plot for overlap

# biased
s = DDf_b_o.loc[DDf_b_o['Type'] == 'biased']
data = s.Overlap.values
data = (data,)
res = bootstrap(data, np.median, confidence_level=0.95, method = "percentile")

biased_median = np.median(data)
biased_ci_low = res.confidence_interval.low
biased_ci_high = res.confidence_interval.high

# unbiased
s = DDf_b_o.loc[DDf_b_o['Type'] == 'unbiased']
data = s.Overlap.values
data = (data,)
res = bootstrap(data, np.median, confidence_level=0.95, method = "percentile")

unbiased_median = np.median(data)
unbiased_ci_low = res.confidence_interval.low
unbiased_ci_high = res.confidence_interval.high

# zebrafish
s = DDf_b_o.loc[DDf_b_o['Type'] == 'zebrafish']
data = s.Overlap.values
data = (data,)
res = bootstrap(data, np.median, confidence_level=0.95, method = "percentile")

zebrafish_median = np.median(data)
zebrafish_ci_low = res.confidence_interval.low
zebrafish_ci_high = res.confidence_interval.high

    
df = pd.DataFrame()
df['Type'] = ['unbiased', 'biased', 'zebrafish']
df['Geodesic Intersection'] = [unbiased_median, biased_median, zebrafish_median]
df['CI_low'] = [unbiased_median - unbiased_ci_low, biased_median - biased_ci_low, zebrafish_median - zebrafish_ci_low]
df['CI_high'] = [unbiased_ci_high - unbiased_median, biased_ci_high - biased_median, zebrafish_ci_high - zebrafish_median]



In [ ]:
df['category'] = [str(i) for i in df.index]
fig = px.bar(df, 
       x="Type", 
       y='Geodesic Intersection', 
       color = "Type", 
       error_y = "CI_high", 
       error_y_minus = "CI_low", 
       color_discrete_map = {'unbiased': '#cd34b5', 'biased': '#0064ab', 'zebrafish': '#ab7ca3'})

fig.update_layout(paper_bgcolor="white",
                 plot_bgcolor="white",
                 showlegend = False,)
#for i in range(3):
#    fig.data[i].error_y.thickness = 12
fig.update_layout(
    width=500, 
    height=500,
    font=dict(size=20)
)
fig.write_image('matroid_GI.svg')

fig.show()

In [ ]:
# bar plot for ROG

# biased
s = Df[Df['Type'] == 'biased']
data = s.ROG.values
data = (data,)
res = bootstrap(data, np.median, confidence_level=0.95, method = "percentile")

biased_median = np.median(data)
biased_ci_low = res.confidence_interval.low
biased_ci_high = res.confidence_interval.high

# unbiased
s = Df.loc[Df['Type'] == 'unbiased']
data = s.ROG.values
data = (data,)
res = bootstrap(data, np.median, confidence_level=0.95, method = "percentile")

unbiased_median = np.median(data)
unbiased_ci_low = res.confidence_interval.low
unbiased_ci_high = res.confidence_interval.high

# zebrafish
s = Df.loc[Df['Type'] == 'zebrafish']
data = s.ROG.values
data = (data,)
res = bootstrap(data, np.median, confidence_level=0.95, method = "percentile")

zebrafish_median = np.median(data)
zebrafish_ci_low = res.confidence_interval.low
zebrafish_ci_high = res.confidence_interval.high

    
df = pd.DataFrame()
df['Type'] = ['unbiased', 'biased', 'zebrafish']
df['Volume'] = [unbiased_median, biased_median, zebrafish_median]
df['CI_low'] = [unbiased_median - unbiased_ci_low, biased_median - biased_ci_low, zebrafish_median - zebrafish_ci_low]
df['CI_high'] = [unbiased_ci_high - unbiased_median, biased_ci_high - biased_median, zebrafish_ci_high - zebrafish_median]



In [ ]:
df['category'] = [str(i) for i in df.index]
fig = px.bar(df, 
       x="Type", 
       y='Volume', 
       color = "Type", 
       error_y = "CI_high", 
       error_y_minus = "CI_low", 
       color_discrete_map = {'unbiased': '#cd34b5', 'biased': '#0064ab', 'zebrafish': '#ab7ca3'})

fig.update_layout(paper_bgcolor="white",
                 plot_bgcolor="white",
                 showlegend = False,)
#for i in range(3):
#    fig.data[i].error_y.thickness = 12
fig.update_layout(
    width=500, 
    height=500,
    font=dict(size=20)
)
fig.write_image('matroid_Volume.svg')

fig.show()

In [ ]:
# bar plot for ROG

# biased
s = Df[Df['Type'] == 'biased']
data = s.loc[:,"Life Span"].values
data = (data,)
res = bootstrap(data, np.median, confidence_level=0.95, method = "percentile")

biased_median = np.median(data)
biased_ci_low = res.confidence_interval.low
biased_ci_high = res.confidence_interval.high

# unbiased
s = Df.loc[Df['Type'] == 'unbiased']
data = s.loc[:,"Life Span"].values
data = (data,)
res = bootstrap(data, np.median, confidence_level=0.95, method = "percentile")

unbiased_median = np.median(data)
unbiased_ci_low = res.confidence_interval.low
unbiased_ci_high = res.confidence_interval.high

# zebrafish
s = Df.loc[Df['Type'] == 'zebrafish']
data = s.loc[:,"Life Span"].values
data = (data,)
res = bootstrap(data, np.median, confidence_level=0.95, method = "percentile")

zebrafish_median = np.median(data)
zebrafish_ci_low = res.confidence_interval.low
zebrafish_ci_high = res.confidence_interval.high

df = pd.DataFrame()
df['Type'] = ['unbiased', 'biased', 'zebrafish']
df['Geodesic Size'] = [unbiased_median, biased_median, zebrafish_median]
df['CI_low'] = [unbiased_median - unbiased_ci_low, biased_median - biased_ci_low, zebrafish_median - zebrafish_ci_low]
df['CI_high'] = [unbiased_ci_high - unbiased_median, biased_ci_high - biased_median, zebrafish_ci_high - zebrafish_median]



In [ ]:
df['category'] = [str(i) for i in df.index]
fig = px.bar(df, 
       x="Type", 
       y='Geodesic Size', 
       color = "Type", 
       error_y = "CI_high", 
       error_y_minus = "CI_low", 
       color_discrete_map = {'unbiased': '#cd34b5', 'biased': '#0064ab', 'zebrafish': '#ab7ca3'})

fig.update_layout(paper_bgcolor="white",
                 plot_bgcolor="white",
                 showlegend = False,)
#for i in range(3):
#    fig.data[i].error_y.thickness = 12
fig.update_layout(
    width=500, 
    height=500,
    font=dict(size=20)
)
fig.write_image('matroid_GS.svg')

fig.show()

In [ ]:
# bar plot for Size

# biased
s = Df[Df['Type'] == 'biased']
data = s.loc[:,"Size"].values
data = (data,)
res = bootstrap(data, np.median, confidence_level=0.95, method = "percentile")

biased_median = np.median(data)
biased_ci_low = res.confidence_interval.low
biased_ci_high = res.confidence_interval.high

# unbiased
s = Df.loc[Df['Type'] == 'unbiased']
data = s.loc[:,"Size"].values
data = (data,)
res = bootstrap(data, np.median, confidence_level=0.95, method = "percentile")

unbiased_median = np.median(data)
unbiased_ci_low = res.confidence_interval.low
unbiased_ci_high = res.confidence_interval.high

# zebrafish
s = Df.loc[Df['Type'] == 'zebrafish']
data = s.loc[:,"Size"].values
data = (data,)
res = bootstrap(data, np.median, confidence_level=0.95, method = "percentile")

zebrafish_median = np.median(data)
zebrafish_ci_low = res.confidence_interval.low
zebrafish_ci_high = res.confidence_interval.high

df = pd.DataFrame()
df['Type'] = ['unbiased', 'biased', 'zebrafish']
df['Size'] = [unbiased_median, biased_median, zebrafish_median]
df['CI_low'] = [unbiased_median - unbiased_ci_low, biased_median - biased_ci_low, zebrafish_median - zebrafish_ci_low]
df['CI_high'] = [unbiased_ci_high - unbiased_median, biased_ci_high - biased_median, zebrafish_ci_high - zebrafish_median]



In [ ]:
df['category'] = [str(i) for i in df.index]
fig = px.bar(df, 
       x="Type", 
       y='Size', 
       color = "Type", 
       error_y = "CI_high", 
       error_y_minus = "CI_low", 
       color_discrete_map = {'unbiased': '#cd34b5', 'biased': '#0064ab', 'zebrafish': '#ab7ca3'})

fig.update_layout(paper_bgcolor="white",
                 plot_bgcolor="white",
                 showlegend = False,)
#for i in range(3):
#    fig.data[i].error_y.thickness = 12
fig.update_layout(
    width=500, 
    height=500,
    font=dict(size=20)
)
fig.write_image('matroid_size.svg')

fig.show()

In [ ]:
DDf_b_o = pd.read_csv('communities_overlap_zebrafish_matroidgeneratos.csv')
Df = pd.read_csv('communities_stat_zebrafish_matroidgeneratos.csv')
Df['Partial ambient ass'] = Df['Size']/Df['ROG']
Df['Partial intrinsic ass'] = Df['Size']/Df['Life Span']

In [33]:
AA,IA = [],[]
CI_lowAA = []
CI_highAA = []
CI_lowIA = []
CI_highIA = []

for Type in ['unbiased', 'zebrafish','biased']:
    s = Df.loc[Df['Type'] == Type]
    r = np.array(s['ROG'])
    w = np.array(s['Size'])
    l = np.array(s['Life Span']) 
    AA.append(np.sum(np.array(s['Partial ambient ass'])/len(s['Partial ambient ass'])))
    IA.append(np.sum(np.array(s['Partial intrinsic ass'])/len(s['Partial intrinsic ass'])))
    
    dataAA = s["Partial ambient ass"]
    dataIA = s["Partial intrinsic ass"]
    
    # compute 95% confidence interval
    ci_lowerAA, ci_upperAA = st.norm.interval(alpha=0.95, loc=np.mean(dataAA), scale=st.sem(dataAA))
    ci_lowerIA, ci_upperIA = st.norm.interval(alpha=0.95, loc=np.mean(dataIA), scale=st.sem(dataIA))
    CI_lowAA.append(ci_lowerAA)
    CI_highAA.append(ci_upperAA)
    CI_lowIA.append(ci_lowerIA)
    CI_highIA.append(ci_upperIA)
    
AS = pd.DataFrame()
AS['Ambient assortativity'] = AA
AS['Intrinsic assortativity'] = IA
AS['Type'] = ['unbiased', 'zebrafish','biased']
AS['CI_lowAA'] = [element1 - element2 for (element1, element2) in zip(AA, CI_lowAA)]
AS['CI_highAA'] = [element1 - element2 for (element1, element2) in zip(CI_highAA, AA)]
AS['CI_lowIA'] = [element1 - element2 for (element1, element2) in zip(IA, CI_lowIA)]
AS['CI_highIA'] = [element1 - element2 for (element1, element2) in zip(CI_highIA, IA)]
AS

,Ambient assortativity,Intrinsic assortativity,Type,CI_lowAA,CI_highAA,CI_lowIA,CI_highIA
0,7.003347,17.163340,unbiased,0.199240,0.199240,0.587129,0.587129
1,8.192485,17.671645,zebrafish,0.420350,0.420350,0.941941,0.941941
2,7.561790,17.545480,biased,0.252189,0.252189,0.586685,0.586685


In [36]:
AS['category'] = [str(i) for i in AS.index]
fig = px.bar(AS, 
       x="Type", 
       y="Ambient assortativity", 
       color = "Type", 
       error_y = "CI_highAA", 
       error_y_minus = "CI_lowAA", 
       color_discrete_map = {'unbiased': '#cd34b5', 'biased': '#0064ab', 'zebrafish': '#ab7ca3'})

fig.update_layout(paper_bgcolor="white",
                 plot_bgcolor="white",
                 showlegend = False,)

fig.update_layout(
    width=500, 
    height=500,
    font=dict(size=20)
)
fig.write_image('AA_matr.svg')

fig.show()


In [37]:
AS['category'] = [str(i) for i in AS.index]
fig = px.bar(AS, 
       x="Type", 
       y="Intrinsic assortativity", 
       color = "Type", 
       error_y = "CI_highIA", 
       error_y_minus = "CI_lowIA", 
       color_discrete_map = {'unbiased': '#cd34b5', 'biased': '#0064ab', 'zebrafish': '#ab7ca3'})

fig.update_layout(paper_bgcolor="white",
                 plot_bgcolor="white",
                 showlegend = False,)

fig.update_layout(
    width=500, 
    height=500,
    font=dict(size=20)
)
fig.write_image('IA_matr.svg')

fig.show()


In [ ]:
B = Df.loc[Df['Type'] == 'biased']
U = Df.loc[Df['Type'] == 'unbiased']
Z = Df.loc[Df['Type'] == 'zebrafish']
BB,UU,ZZ = [],[],[]



for M in['Size','ROG','Life Span']:
    for el in B[M]:
        BB.append(el)
    for el in U[M]:
        UU.append(el)
    for el in Z[M]:
        ZZ.append(el)
    



B = DDf_b_o.loc[DDf_b_o['Type'] == 'biased']
U = DDf_b_o.loc[DDf_b_o['Type'] == 'unbiased']
Z = DDf_b_o.loc[DDf_b_o['Type'] == 'zebrafish']
M = 'Overlap'

for el in B[M]:
    BB.append(el)
for el in U[M]:
    UU.append(el)
for el in Z[M]:
    ZZ.append(el)

M,N = np.zeros((3,3)),np.zeros((3,3))
ALL = [UU,BB,ZZ]
for a in ALL:
    for b in ALL:
        M[ALL.index(a),ALL.index(b)] = mannwhitneyu(a,b, alternative = 'two-sided')[1]
        N[ALL.index(a),ALL.index(b)]= kstest(a,b)[1]



### Statistical tests on communities distributions: 
1) Significant differences between unbiased and biased tracks

2) Significant differences between unbiased and zebrafish tracks.

3) The hypothesis that biased and zebrafish tracks are different is rejected.

In [ ]:
fig = px.imshow(M, color_continuous_scale = 'purples',
          text_auto='.2e')
fig.update_coloraxes(showscale=False)
fig.update_layout(yaxis = dict(
        tickmode = 'array',
        tickvals = [0,1,2,3,4],
    
        ticktext = ['Unbiased','Biased','Zebrafish']
    ),
        xaxis = dict(
        tickmode = 'array',
        tickvals = [0,1,2,3,4],
    
        ticktext = ['Unbiased','Biased','Zebrafish']
    ))

fig.show()

In [ ]:
fig = px.imshow(N, color_continuous_scale = 'purples',
          text_auto='.2e')
fig.update_coloraxes(showscale=False)
fig.update_layout(yaxis = dict(
        tickmode = 'array',
        tickvals = [0,1,2,3,4],
    
        ticktext = ['Unbiased','Biased','Zebrafish']
    ),
        xaxis = dict(
        tickmode = 'array',
        tickvals = [0,1,2,3,4],
    
        ticktext = ['Unbiased','Biased','Zebrafish']
    ))


fig.write_image('heat_matroid.svg')

fig.show()

# Nematodes trajectories

In [ ]:
for i in range(6):
    fig = plot_chain_with_centrality('nematode_{}'.format(i),'nematodes')
    fig.write_image('nematode_{}_matr.svg'.format(i))
    fig.show()

In [ ]:
for i in range(6):
    fig = plot_chain_with_communities('nematode_{}'.format(i),'nematodes')
    fig.write_image('nematode_{}_matr_comms.svg'.format(i))
    fig.show()